In [1]:
import autogen

# LLM Configuration (same as in the original notebook)
llm_config = {
    "model": "gpt-3.5-turbo"
}

In [2]:
import os
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [3]:
# Customer Proxy Agent
customer_proxy_agent = autogen.ConversableAgent(
    name="Customer_Proxy", 
    system_message="Communicates customer's requirements to the Project Manager.",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

# Project Manager Agent
project_manager_agent = autogen.ConversableAgent(
    name="Project_Manager",  
    system_message="Receives customer requirements and creates project plan based on outputs of each engineer.",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

# Requirement Engineer Agent
requirement_engineer_agent = autogen.ConversableAgent(
    name="Requirement_Engineer",  
    system_message="""First, creates the requirements document that has the requirements for the use-cases of the project requested by the project manager.
                      Then calculate the {effort} to create the requirements document in hours, days, weeks, or months, follow the steps below:
                          - step 1. work = {the estimate of total number of requirements for the use-cases needed for the software project} 
                          - step 2. productivity rate = {5 requirements completed every day} 
                          - step 3. effort = {work}/{productivity} .
                          
                      The amount of {work} in this phase is measured by the total number of requirements of all use-cases that are created and documented for the software project.""",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

# System Engineer Agent
system_engineer_agent = autogen.ConversableAgent(
    name="System_Engineer",  
    system_message="""First, creates the detailed design document based on the requirements document created by the requirement engineer for the requested software project. 
                      The System Engineer estimates the number of pages for each design element in the design document based on the number of requirements in the requirements document.
                      Then, calculate the {effort} to create the design document in hours, days, weeks, or months, follow the steps below:
                          - step 1. work = {the estimate of total number of pages in the design document for the software project} 
                          - step 2. productivity rate = {5 pages completed every day} 
                          - step 3. effort = {work}/{productivity} .
                          
                      The amount of {work} in this phase is measured by the total number of pages in the design document that are created 
                      based on the total number of requirements in the requirements document created by the requirement engineer for the software project.""",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

# Software Engineer Agent
software_engineer_agent = autogen.ConversableAgent(
    name="Software_Engineer",  
    system_message="""First, writes complete source code step by step and develops the software based on each design element of the design document created by the system engineer 
                      and requirements document by the requirement engineer.
                      The Software Engineer estimates the number of source lines of code (SLOC) based on the design elements in the design document.
                      Then, calculate the {effort} to write the source code (SLOC) in hours, days, weeks, or months, follow the steps below:
                          - step 1. work = {the estimate of total number of lines of code (SLOC) that will be created for the software project} 
                          - step 2. productivity rate = {50 SLOC completed every day} 
                          - step 3. effort = {work}/{productivity} .
                          
                      The amount of {work} in this phase is measured by the total number of lines of code created based on the 
                      total number of pages in the provided design document for the software project.""",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

# Test Engineer Agent
test_engineer_agent = autogen.ConversableAgent(
    name="Test_Engineer",  
    system_message="""First, creates the detailed test plan document which has all test cases and executes the test cases developed based on the software written by the software engineer, 
                      design document created by the system engineer, and requirements document created by the requirement engineer.
                      The Test Engineer estimates the number of test cases in the test plan based on the number of requirements in the requirements document.
                      Then, calculate the {effort} to write and execute the test cases in the test plan in hours, days, weeks, or months, follow the steps below:
                          - step 1. work = {the estimate of total number of test cases that will be created and executed for the software project} 
                          - step 2. productivity rate = {2 test cases completed every day} 
                          - step 3. effort = {work}/{productivity} .
                          
                      The amount of {work} in this phase is measured by the total number of test cases created and executed based on the 
                      total number of requirements in the provided requirements specification document for the software project.""",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

# Documentation Engineer Agent
documentation_engineer_agent = autogen.ConversableAgent(
    name="Documentation_Engineer",  
    system_message="""First, creates the detailed documentation which includes user documentation and training material based on the software written by the software engineer, 
                      design document created by the system engineer, and requirements document created by the requirement engineer. 
                      The Documentation Engineer estimates the number of pages for the documentation based on the number of requirements in the requirements document, 
                      number of pages in the design document, and total number of SLOC written.
                      Then, calculate the {effort} to write documentation pages in the documentation and training phase in hours, days, weeks, or months, follow the steps below:
                          - step 1. work = {the estimate of total number of documentation pages that will be created for the software project} 
                          - step 2. productivity rate = {3 pages completed every day} 
                          - step 3. effort = {work}/{productivity} .
                          
                      The amount of {work} in this phase is measured by the total number of documentation pages based on the 
                      total number of requirements in the provided requirements specification document for the software project.""",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)


In [4]:
# Initial message from Customer Proxy to Project Manager
customer_message = (
    "I want to create a fully automated DevSecOps pipeline for our organization. "
    "The pipeline should handle continuous integration, delivery, and deployment, integrate security tools like SAST/DAST, "
    "automate infrastructure provisioning using Infrastructure-as-Code (IaC), and ensure compliance and audit logging throughout the process."
)

# Create a list of agent pairs to define the conversation flow
conversation_flow = [
    # Customer Proxy to Project Manager
    (customer_proxy_agent, project_manager_agent, customer_message),

    # Project Manager to Requirement Engineer
    (project_manager_agent, requirement_engineer_agent, 
     "I have received the customer requirements from the Customer Proxy. Please list the detailed use cases and requirements for automating "
     "a DevSecOps pipeline. Make sure to include use cases for CI/CD pipeline automation, security integration, infrastructure-as-code (IaC), and compliance tracking. "
     "Ensure clarity and completeness in defining each requirement. "
     "Please create at least 10 use cases, with a unique name and number for each requirement (e.g., REQ-01, REQ-02) across all use cases. "
     "Also, provide detailed estimates for the amount of work, effort, and productivity required to complete the requirements phase, measured by how many use cases and requirements are documented. "
     "Once completed, report back to me so I can proceed with the project plan."),

    # Requirement Engineer to Project Manager after gathering requirements
    (requirement_engineer_agent, project_manager_agent, 
     "I have documented the use cases and numbered each requirement as requested. "
     "I have provided the work and effort estimates along with the productivity rate for completing the requirements phase, based on the complexity of the DevSecOps automation requirements. "
     "I am now handing this over to you to proceed with the next steps in the project plan."),

    # Project Manager to System Engineer
    (project_manager_agent, system_engineer_agent, 
     "The Requirements Engineer has completed documenting the requirements for automating the DevSecOps pipeline. "
     "Please proceed with creating the detailed design document that includes the architecture for the CI/CD pipeline, security integration, and infrastructure automation based on the requirements document. "
     "Also provide detailed estimates for the amount of work, effort, and productivity required to complete the system design phase, measured by the number of design elements and diagrams created. "
     "Once completed, report back to me so I can proceed with the next steps in the project plan."),

    # System Engineer to Project Manager after design phase
    (system_engineer_agent, project_manager_agent, 
     "I have completed the design document based on the provided requirements for the DevSecOps pipeline. "
     "I have provided the work and effort estimates along with the productivity rate specified for the design phase, based on the number of architectural elements. "
     "I am now handing this over back to you to proceed with the next steps in the project plan."),

    # Project Manager to Software Engineer
    (project_manager_agent, software_engineer_agent, 
     "The System Engineer has completed the design of the DevSecOps pipeline. "
     "Please proceed with writing the source code to develop the pipeline automation for CI/CD, security testing integration, and infrastructure provisioning using the provided design. "
     "Also provide detailed estimates for the amount of work, effort, and productivity required for the development phase, measured by the number of lines of code (SLOC) written. "
     "Once completed, report back to me so we can proceed with the next steps in the project plan."),

    # Software Engineer to Project Manager after development phase
    (software_engineer_agent, project_manager_agent, 
     "I have completed the software development for automating the DevSecOps pipeline, including CI/CD, security integration, and infrastructure automation. "
     "I have provided the work and effort estimates along with the productivity rate specified for the implementation phase, based on the lines of code (SLOC) written. "
     "I am now handing this over back to you to proceed with the next steps in the project plan."),

    # Project Manager to Test Engineer
    (project_manager_agent, test_engineer_agent, 
     "The software has been developed. "
     "Please proceed with creating the detailed test plan document which includes all test cases and executes the test cases developed based on the software written by the software engineer, "
     "the design document created by the system engineer, and the requirements document created by the requirement engineer. "
     "Also provide detailed estimates for the amount of work, effort, and productivity required to complete the testing phase, measured by the number of test cases executed. "
     "Once completed, report back to me so we can proceed with the next steps in the project plan."),

    # Test Engineer to Project Manager after testing phase
    (test_engineer_agent, project_manager_agent, 
     "I have completed the testing phase and verified that the software meets the requirements. "
     "I have provided the work and effort estimates along with the productivity rate specified for the testing phase. "
     "I am now handing this over back to you to proceed with the next steps in the project plan."),

    # Project Manager to Documentation Engineer
    (project_manager_agent, documentation_engineer_agent, 
     "The testing is complete. "
     "Please proceed with creating the detailed documentation, which includes user documentation and training material based on the software written by the software engineer, "
     "the design document created by the system engineer, and the requirements document created by the requirement engineer. "
     "Also provide detailed estimates for the amount of work, effort, and productivity required for the documentation phase, measured by the number of pages created. "
     "Once completed, report back to me so we can finalize the project."),

    # Documentation Engineer to Project Manager after documentation phase
    (documentation_engineer_agent, project_manager_agent, 
     "I have completed the documentation, including user documentation and training material. "
     "I have provided the work and effort estimates along with the productivity rate specified for the documentation phase. "
     "I am now handing this over back to you to proceed with finalizing the project.")
]


In [13]:
# Initialize GroupChat using Round Robin speaker selection method
groupchat = autogen.GroupChat(
    agents=[
        customer_proxy_agent,
        project_manager_agent,
        requirement_engineer_agent,
        system_engineer_agent,
        software_engineer_agent,
        test_engineer_agent,
        documentation_engineer_agent,
    ],
    messages=[],
    speaker_selection_method="round_robin",  # Automatically selects the next speaker
    max_round=20,
)

# Set up the GroupChatManager to manage the conversation
manager = autogen.GroupChatManager(
    groupchat=groupchat, llm_config=llm_config
)

In [15]:
# Start the conversation with Customer Proxy sending the initial message to Project Manager
groupchat_result = customer_proxy_agent.initiate_chat(
    recipient=project_manager_agent,
    message=customer_message,
)

# Project Manager initiates the conversation with the Requirement Engineer
groupchat_result = project_manager_agent.initiate_chat(
    recipient=requirement_engineer_agent,
    message=(
        "I have received the customer requirements from the Customer Proxy. Please list the detailed use cases and requirements for automating "
        "a DevSecOps pipeline. Make sure to include use cases for CI/CD pipeline automation, security integration, infrastructure-as-code (IaC), and compliance tracking. "
        "Ensure clarity and completeness in defining each requirement. "
        "Please create at least 10 use cases, with a unique name and number for each requirement (e.g., REQ-01, REQ-02) across all use cases. "
        "Also, provide detailed estimates for the amount of work, effort, and productivity required to complete the requirements phase, measured by how many use cases and requirements are documented. "
        "Once completed, report back to me so I can proceed with the project plan."
    )
)


Customer_Proxy (to Project_Manager):

I want to create a fully automated DevSecOps pipeline for our organization. The pipeline should handle continuous integration, delivery, and deployment, integrate security tools like SAST/DAST, automate infrastructure provisioning using Infrastructure-as-Code (IaC), and ensure compliance and audit logging throughout the process.

--------------------------------------------------------------------------------


Replying as Project_Manager. Provide feedback to Customer_Proxy. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
Project_Manager (to Customer_Proxy):

To create a fully automated DevSecOps pipeline for your organization, we will need the expertise of several engineers specializing in different areas. Here is a high-level project plan outlining the key tasks each engineer will be responsible for:

1. **Infrastructure Engineer:**
   - Design and implement the infrastructure provisioning using Infrastructure-as-Code (IaC) tools like Terraform or CloudFormation.
   - Ensure scalability, reliability, and security of the infrastructure.

2. **Security Engineer:**
   - Integrate security tools like Static Application Security Testing (SAST) and Dynamic Application Security Testing (DAST) into the pipeline.
   - Configure security scanners and define scanning policies to ensure vulnerability assessments at every stage.

3. **CI/CD Engineer:**
   - Setup and configure the Continuous Integration/Continuous Deployment (CI/CD) pipeline using to

Replying as Customer_Proxy. Provide feedback to Project_Manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
Customer_Proxy (to Project_Manager):

I have communicated your requirement for a fully automated DevSecOps pipeline to the Product Owner. The project plan outlines the key tasks that each specialized engineer will be responsible for. If you have any specific preferences or additional requirements, please let me know so that we can incorporate them into the plan.

--------------------------------------------------------------------------------


...

Replying as Project_Manager. Provide feedback to Requirement_Engineer. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
Project_Manager (to Requirement_Engineer):

Thank you for providing the detailed use cases and requirements for automating the DevSecOps pipeline. The effort estimation for documenting these requirements is around 20-30 hours, considering the need for clarity and completeness in each item.

Based on the outputs provided by you as the Requirement Engineer, I will now proceed with creating the project plan, including the design and implementation phases for the DevSecOps pipeline. This plan will outline the timelines, resources, and tasks required to achieve the project goals effectively.

I will collaborate with the rest of the team, including the Development Engineer, Security Engineer, Infrastructure Engineer, and Compliance Engineer, to ensure a holistic approach to the project plan. I aim to create a roadmap that aligns with the documented requirements and addresses the customer's needs effectively.

Please stand by fo

Replying as Requirement_Engineer. Provide feedback to Project_Manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
Requirement_Engineer (to Project_Manager):

That sounds like a great plan! I'm glad I could assist in defining the requirements for the DevSecOps pipeline. Collaborating with the team members representing various expertise areas will ensure a comprehensive and effective project plan. I'm excited to see the finalized project plan and witness the transformation of the DevSecOps pipeline.

If you need any further assistance during the implementation phase or require any additional clarification on the requirements, feel free to reach out. Let's indeed automate that DevSecOps pipeline and deliver a robust and secure solution for our customer! Good luck with the project planning, and I'm here to support you as needed.

--------------------------------------------------------------------------------


Replying as Project_Manager. Provide feedback to Requirement_Engineer. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  exit


In [ ]:
# Define agents with Scrum roles and specific responsibilities
# Customer Proxy Agent
customer_proxy_agent = autogen.ConversableAgent(
    name="Customer_Proxy",
    system_message="Communicates customer's requirements to the Product Owner.",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

# Product Owner Agent
product_owner_agent = autogen.ConversableAgent(
    name="Product_Owner",
    system_message="Defines and prioritizes the backlog items and requirements for the DevSecOps pipeline. Sets goals for each sprint.",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

# Scrum Master Agent
scrum_master_agent = autogen.ConversableAgent(
    name="Scrum_Master",
    system_message="Facilitates sprint planning, daily standups, sprint reviews, and retrospectives. Ensures the team focuses on sprint goals.",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

# Requirement Engineer Agent
requirement_engineer_agent = autogen.ConversableAgent(
    name="Requirement_Engineer",
    system_message="""Documents requirements for the DevSecOps pipeline. Estimates effort for documenting use cases based on productivity rates.""",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

# System Engineer Agent
system_engineer_agent = autogen.ConversableAgent(
    name="System_Engineer",
    system_message="""Creates a design document for the DevSecOps pipeline based on requirements. Estimates work based on the number of design elements.""",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

# Software Engineer Agent
software_engineer_agent = autogen.ConversableAgent(
    name="Software_Engineer",
    system_message="""Develops software for the DevSecOps pipeline, focusing on CI/CD, security, and IaC components. Provides effort estimation for the development phase.""",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

# Test Engineer Agent
test_engineer_agent = autogen.ConversableAgent(
    name="Test_Engineer",
    system_message="""Develops and executes test cases based on project requirements and design documents. Estimates effort based on productivity.""",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

# Documentation Engineer Agent
documentation_engineer_agent = autogen.ConversableAgent(
    name="Documentation_Engineer",
    system_message="""Creates user documentation and training material based on project specifications. Estimates effort based on documentation size.""",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

# Release Manager Agent
release_manager_agent = autogen.ConversableAgent(
    name="Release_Manager",
    system_message="Coordinates deployment and release activities, ensuring readiness for production.",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

# Architect Agent
architect_agent = autogen.ConversableAgent(
    name="Architect",
    system_message="Designs high-level system architecture, ensuring scalability, performance, and security of the DevSecOps pipeline.",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

# Business Analyst Agent
business_analyst_agent = autogen.ConversableAgent(
    name="Business_Analyst",
    system_message="Gathers and documents business requirements to align with organizational goals.",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

# Define the initial message and Scrum-specific messages for the sprint cycle
initial_message = (
    "I want to create a fully automated DevSecOps pipeline for our organization. "
    "The pipeline should handle CI/CD, security tool integration (SAST/DAST), IaC automation, and compliance tracking."
)

# Conversation flow for Scrum sprint cycle using round-robin
sprint_conversation_flow = [
    # Product Owner to Scrum Master
    (product_owner_agent, scrum_master_agent, "This sprint, focus on implementing CI/CD setup, security integration, and initial infrastructure provisioning."),

    # Scrum Master to Requirement Engineer
    (scrum_master_agent, requirement_engineer_agent, "Document requirements and estimate effort for DevSecOps pipeline use cases."),

    # Scrum Master to System Engineer
    (scrum_master_agent, system_engineer_agent, "Create the design document based on the documented requirements."),

    # Scrum Master to Software Engineer
    (scrum_master_agent, software_engineer_agent, "Develop the initial software modules for CI/CD integration."),

    # Scrum Master to Test Engineer
    (scrum_master_agent, test_engineer_agent, "Create and execute test cases based on initial software modules."),

    # Scrum Master to Documentation Engineer
    (scrum_master_agent, documentation_engineer_agent, "Create initial user documentation and training materials."),

    # Scrum Master to Release Manager
    (scrum_master_agent, release_manager_agent, "Prepare components for release to production."),

    # Scrum Master to Architect
    (scrum_master_agent, architect_agent, "Design high-level architecture to ensure scalability and security."),

    # Scrum Master to Business Analyst
    (scrum_master_agent, business_analyst_agent, "Gather business requirements from stakeholders for alignment with project goals."),
]

# Initialize the GroupChat using round-robin with the new Scrum roles
groupchat = autogen.GroupChat(
    agents=[
        customer_proxy_agent,
        product_owner_agent,
        scrum_master_agent,
        requirement_engineer_agent,
        system_engineer_agent,
        software_engineer_agent,
        test_engineer_agent,
        documentation_engineer_agent,
        release_manager_agent,
        architect_agent,
        business_analyst_agent,
    ],
    messages=[],
    speaker_selection_method="round_robin",  # Automatically selects the next speaker
    max_round=20,
)

# Set up the GroupChatManager to manage the conversation
manager = autogen.GroupChatManager(
    groupchat=groupchat, llm_config=llm_config
)

# Step 1: Customer Proxy communicates initial requirements to Product Owner
customer_proxy_message = (
    "Our goal is to implement a fully automated DevSecOps pipeline, which includes CI/CD, security integrations (SAST/DAST), IaC automation, and compliance tracking."
)
groupchat_result = customer_proxy_agent.initiate_chat(
    recipient=product_owner_agent,
    message=customer_proxy_message
)

# Step 2: Product Owner defines the sprint focus and relays it to Scrum Master
sprint_goal_message = (
    "This sprint, focus on setting up the initial CI/CD pipeline, integrating security tools, and establishing foundational infrastructure for IaC."
)
groupchat_result = product_owner_agent.initiate_chat(
    recipient=scrum_master_agent,
    message=sprint_goal_message
)

# Define the enhanced sprint conversation flow with Daily Standup, Sprint Review, and Sprint Retrospective
enhanced_sprint_conversation_flow = [
    # Sprint Planning and Task Delegation
    (scrum_master_agent, requirement_engineer_agent, "Document requirements and estimate effort for DevSecOps pipeline use cases."),
    (scrum_master_agent, system_engineer_agent, "Create the design document based on the documented requirements."),
    (scrum_master_agent, software_engineer_agent, "Develop the initial software modules for CI/CD integration."),
    (scrum_master_agent, test_engineer_agent, "Create and execute test cases based on initial software modules."),
    (scrum_master_agent, documentation_engineer_agent, "Create initial user documentation and training materials."),
    (scrum_master_agent, release_manager_agent, "Prepare components for release to production."),
    (scrum_master_agent, architect_agent, "Design high-level architecture to ensure scalability and security."),
    (scrum_master_agent, business_analyst_agent, "Gather business requirements from stakeholders for alignment with project goals."),

    # Daily Standup (each team member shares updates)
    (scrum_master_agent, requirement_engineer_agent, "Daily Standup: Share progress, blockers, and next steps."),
    (scrum_master_agent, system_engineer_agent, "Daily Standup: Share progress, blockers, and next steps."),
    (scrum_master_agent, software_engineer_agent, "Daily Standup: Share progress, blockers, and next steps."),
    (scrum_master_agent, test_engineer_agent, "Daily Standup: Share progress, blockers, and next steps."),
    (scrum_master_agent, documentation_engineer_agent, "Daily Standup: Share progress, blockers, and next steps."),
    (scrum_master_agent, release_manager_agent, "Daily Standup: Share progress, blockers, and next steps."),
    (scrum_master_agent, architect_agent, "Daily Standup: Share progress, blockers, and next steps."),
    (scrum_master_agent, business_analyst_agent, "Daily Standup: Share progress, blockers, and next steps."),

    # Sprint Review (Scrum Master and Product Owner review completed work)
    (scrum_master_agent, product_owner_agent, "Sprint Review: Present completed work and gather feedback from stakeholders."),

    # Sprint Retrospective (each team member reflects on the sprint)
    (scrum_master_agent, requirement_engineer_agent, "Retrospective: Share what went well, challenges faced, and areas for improvement."),
    (scrum_master_agent, system_engineer_agent, "Retrospective: Share what went well, challenges faced, and areas for improvement."),
    (scrum_master_agent, software_engineer_agent, "Retrospective: Share what went well, challenges faced, and areas for improvement."),
    (scrum_master_agent, test_engineer_agent, "Retrospective: Share what went well, challenges faced, and areas for improvement."),
    (scrum_master_agent, documentation_engineer_agent, "Retrospective: Share what went well, challenges faced, and areas for improvement."),
    (scrum_master_agent, release_manager_agent, "Retrospective: Share what went well, challenges faced, and areas for improvement."),
    (scrum_master_agent, architect_agent, "Retrospective: Share what went well, challenges faced, and areas for improvement."),
    (scrum_master_agent, business_analyst_agent, "Retrospective: Share what went well, challenges faced, and areas for improvement."),
]

# Set up the GroupChatManager to manage the conversation
manager = autogen.GroupChatManager(
    groupchat=groupchat, llm_config=llm_config
)

# Execute the enhanced conversation flow
for i, (sender, recipient, message) in enumerate(enhanced_sprint_conversation_flow):
    # Display next speaker
    if i < len(enhanced_sprint_conversation_flow):
        next_speaker = enhanced_sprint_conversation_flow[i][0].name.replace("_", " ")
        print(f"Next Speaker: {next_speaker}")
    
    # Initiate chat between sender and recipient in the Scrum cycle
    groupchat_result = manager.initiate_chat(
        agent=sender,
        recipient=recipient,
        message=message
    )


Customer_Proxy (to Product_Owner):

Our goal is to implement a fully automated DevSecOps pipeline, which includes CI/CD, security integrations (SAST/DAST), IaC automation, and compliance tracking.

--------------------------------------------------------------------------------


Replying as Product_Owner. Provide feedback to Customer_Proxy. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
Product_Owner (to Customer_Proxy):

Given the goal of implementing a fully automated DevSecOps pipeline, the backlog items and requirements can be organized as follows:

**Priority 1: Setting up the CI/CD Pipeline**
1.1 Configure version control system (e.g., Git) integration with the pipeline
1.2 Implement continuous integration tools (e.g., Jenkins, GitLab CI/CD)
1.3 Automate the build, test, and deployment processes for all application code
1.4 Establish a process for automated code reviews and quality gates

**Priority 2: Integrating Security Tools (SAST/DAST)**
2.1 Integrate Static Application Security Testing (SAST) tools (e.g., SonarQube, Checkmarx)
2.2 Implement Dynamic Application Security Testing (DAST) tools (e.g., OWASP ZAP, Burp Suite)
2.3 Configure security scans to run as part of the pipeline with automatic fail/success criteria

**Priority 3: Automating Infrastructure as Code (IaC)**
3.1 Set up Infrastruct

Replying as Customer_Proxy. Provide feedback to Product_Owner. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  exi


Customer_Proxy (to Product_Owner):

exi

--------------------------------------------------------------------------------


...

Next Speaker: Scrum Master
chat_manager (to Product_Owner):

Sprint Review: Present completed work and gather feedback from stakeholders.

--------------------------------------------------------------------------------


Replying as Product_Owner. Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  exit


Next Speaker: Scrum Master
chat_manager (to Requirement_Engineer):

Retrospective: Share what went well, challenges faced, and areas for improvement.

--------------------------------------------------------------------------------


Replying as Requirement_Engineer. Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  exit


Next Speaker: Scrum Master
chat_manager (to System_Engineer):

Retrospective: Share what went well, challenges faced, and areas for improvement.

--------------------------------------------------------------------------------
